![RBM](https://pbs.twimg.com/media/Fc9W8bOWAAI_bTY?format=jpg&name=medium)

What makes RBMs different from Boltzmann machines is that visible node isn’t connected to each other, and hidden nodes aren’t connected with each other. Other than that, RBMs are exactly the same as Boltzmann machines

[Click to see my complete article on Restricted Boltzmann Machine](https://medium.com/machine-learning-researcher/boltzmann-machine-c2ce76d94da5)

![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## <font color = #950CDF> Part 1: </font> <font color = #4854E8> Data Preprocessing </font>

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

#### <font color = blue>Import the Libraries

In [91]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

#### <font color = blue> Importing the dataset

In [92]:
movies = pd.read_csv('./ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('./ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('./ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [93]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [94]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [95]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


#### <font color = blue> Preparing the training set and the test set

In [96]:
training_set = pd.read_csv('./ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('./ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [97]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]], shape=(79999, 4))

In [98]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]], shape=(19999, 4))

#### <font color = blue> Getting the number of users and movies

In [99]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

#### <font color = blue> Converting the data into an array with users in lines and movies in columns

In [100]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

#### <font color = blue> Converting the data into Torch tensors

In [101]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

#### <font color = blue> Converting the ratings into binary ratings 1 (Liked) or 0 (Not Liked)

In [102]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## <font color = #950CDF> Part 2: </font> <font color = #4854E8> Build the RBM Model </font>

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

#### <font color = blue> Creating the architecture of the Neural Network

In [103]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        
        # Update weights and biases
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)
nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

## <font color = #950CDF> Part 3: </font> <font color = #4854E8> Training and Testing the Model </font>

![ML](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)

#### <font color = blue> Training the Model

In [104]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(0.3385)
epoch: 2 loss: tensor(0.2492)
epoch: 3 loss: tensor(0.2504)
epoch: 4 loss: tensor(0.2486)
epoch: 5 loss: tensor(0.2481)
epoch: 6 loss: tensor(0.2491)
epoch: 7 loss: tensor(0.2459)
epoch: 8 loss: tensor(0.2525)
epoch: 9 loss: tensor(0.2457)
epoch: 10 loss: tensor(0.2490)


In [137]:
def predict_ratings(rbm, test_data):
    """
    Predicts ratings for the test set using the trained RBM.
    Args:
        rbm: Trained RBM model.
        test_data: Test set (binary ratings, -1 for unrated).
    Returns:
        predicted_ratings: Predicted binary ratings (0 or 1).
    """
    predicted_ratings = []
    for user_ratings in test_data:
        user_ratings = user_ratings.unsqueeze(0)  # Shape: (1, n_movies)
        _, hidden_probs = rbm.sample_h(user_ratings)
        _, predicted = rbm.sample_v(hidden_probs)
        predicted_ratings.append(predicted.squeeze(0))
    return torch.stack(predicted_ratings)

# Get predicted ratings
predicted_test_ratings = predict_ratings(rbm, test_set)

In [138]:
def get_recommendations(user_id, test_set, predicted_ratings, movies_df, n_recommendations=5):
    """
    Get top movie recommendations for a user.
    Args:
        user_id: Index of the user (0-based).
        test_set: Original test set (to filter unrated movies).
        predicted_ratings: Predicted ratings from RBM.
        movies_df: DataFrame containing movie names.
        n_recommendations: Number of recommendations to return.
    Returns:
        DataFrame of recommended movies.
    """
    # Get the user's actual test ratings (0 or 1)
    actual_ratings = test_set[user_id]
    # Get the user's predicted ratings (0 or 1)
    predicted = predicted_ratings[user_id]
    
    # Find movies that were rated in the test set (i.e., not -1)
    rated_mask = (actual_ratings >= 0)
    rated_movies = torch.where(rated_mask)[0]  # Indices of rated movies
    
    # Find movies predicted as "liked" (rating = 1) but not already rated highly
    liked_mask = (predicted == 1)
    recommended_movies = torch.where(liked_mask)[0]
    
    # Exclude movies already rated in the test set
    recommended_movies = [movie for movie in recommended_movies if movie not in rated_movies]
    
    # Get top N recommendations
    top_movies = recommended_movies[:n_recommendations]
    
    # Map movie IDs to titles
    recommendations = []
    for movie_id in top_movies:
        movie_id = movie_id.item() + 1  # Convert to 1-based ID
        movie_title = movies_df[movies_df[0] == movie_id][1].values[0]
        recommendations.append({"Movie ID": movie_id, "Title": movie_title})
    
    return pd.DataFrame(recommendations)

In [145]:
# Example: Get recommendations for user 0
user_id = 50  # First user in the test set
recommendations = get_recommendations(user_id, test_set, predicted_test_ratings, movies)
print("Top 5 Recommended Movies for User", user_id + 1)
print(recommendations)

Top 5 Recommended Movies for User 51
   Movie ID                               Title
0         1                    Toy Story (1995)
1         4            Waiting to Exhale (1995)
2         5  Father of the Bride Part II (1995)
3         7                      Sabrina (1995)
4         8                 Tom and Huck (1995)


#### <font color = blue> Test the Model

In [150]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.2343)


![Machine Learning Project](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

<b>©</b>Amir Ali